In [ ]:
%load_ext watermark


In [ ]:
import itertools as it
import os

from downstream import dstream
import polars as pl
import seaborn as sns
from teeplot import teeplot as tp

import pylib  # noqa: F401


In [ ]:
%watermark -diwmuv -iv


In [ ]:
teeplot_subdir = os.environ.get(
    "NOTEBOOK_NAME", "2024-10-20-qos-dstream-vs-naive-steady"
)
teeplot_subdir


In [ ]:
buffer_size = 64
num_items = 10_000


## Define


In [ ]:
def calc_max_gaps_naive_greedy(
    buffer_size: int, num_ingests: int
) -> list[int]:
    segment_lengths = []
    max_gaps = []
    for i in range(num_ingests):
        segment_lengths.append(1)
        if len(segment_lengths) > buffer_size:
            target = min(
                reversed(range(buffer_size)),
                key=lambda i: segment_lengths[i] + segment_lengths[i + 1],
            )
            segment_lengths[target] += segment_lengths[target + 1]
            segment_lengths.pop(target + 1)
        max_gaps.append(max(segment_lengths) - 1)
        assert sum(segment_lengths) == i + 1
    return max_gaps


In [ ]:
def calc_max_gaps_naive_zhao_2006(
    buffer_size: int, num_ingests: int
) -> list[int]:
    segment_lengths = []
    max_gaps = []
    for i in range(num_ingests):
        if len(segment_lengths) < buffer_size:
            segment_lengths.append(1)
            max_gaps.append(max(segment_lengths) - 1)
            continue
        if (
            len(segment_lengths) > 2
            and segment_lengths[-1] < segment_lengths[-2]
        ):
            segment_lengths[-1] += 1
        else:
            target = min(
                range(buffer_size - 1),
                key=lambda i: segment_lengths[i] + segment_lengths[i + 1],
            )
            segment_lengths[target] += segment_lengths[target + 1]
            segment_lengths.pop(target + 1)
            segment_lengths.append(1)

        assert sum(segment_lengths) == i + 1
        max_gaps.append(max(segment_lengths) - 1)
    return max_gaps


In [ ]:
def calc_max_gaps_dstream(buffer_size: int, num_items: int) -> list[int]:
    return [
        max(
            b - a - 1
            for a, b in it.pairwise(
                [
                    *sorted(
                        dstream.steady_algo.lookup_ingest_times_eager(
                            buffer_size, i + 1
                        ),
                    ),
                    i,
                ],
            )
        )
        if i >= buffer_size
        else 0
        for i in range(num_items)
    ]


## Example Plot


In [ ]:
df = pl.concat(
    [
        pl.DataFrame(
            {
                "Algorithm": "naive steady Zhao06",
                "Max Gap Size": calc_max_gaps_naive_zhao_2006(
                    buffer_size, num_items
                ),
                "Num Items Ingested": range(num_items),
            },
        ),
        pl.DataFrame(
            {
                "Algorithm": "naive steady greedy",
                "Max Gap Size": calc_max_gaps_naive_greedy(
                    buffer_size, num_items
                ),
                "Num Items Ingested": range(num_items),
            },
        ),
        pl.DataFrame(
            {
                "Algorithm": "dstream steady",
                "Max Gap Size": calc_max_gaps_dstream(buffer_size, num_items),
                "Num Items Ingested": range(num_items),
            },
        ),
    ],
)


In [ ]:
with tp.teed(
    sns.relplot,
    df,
    x="Num Items Ingested",
    y="Max Gap Size",
    hue="Algorithm",
    style="Algorithm",
    aspect=2,
    kind="line",
    height=2.2,
    teeplot_subdir=teeplot_subdir,
) as g:
    sns.move_legend(
        g,
        "lower center",
        bbox_to_anchor=(0.4, 1),
        ncol=2,
        title=None,
        frameon=False,
    )
